In [1]:
%matplotlib widget
from util import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from matplotlib import colors
from random import choice
from experiment_class import Experiment,clean_exp_with_hyphaes
from hyphae_id import clean_and_relabel, get_mother, save_hyphaes

In [2]:
plate = 13
directory='//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[1:24]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]

In [3]:
exp= Experiment(13)
exp.load(dates,local=False,raw=False)

In [4]:
exp_clean = clean_and_relabel(exp)

0.0
0.043898156277436345
0.08779631255487269
0.13169446883230904
0.17559262510974538
0.21949078138718173
0.2633889376646181
0.30728709394205445
0.35118525021949076
0.39508340649692714
0.43898156277436345
0.4828797190517998
0.5267778753292361
0.5706760316066726
0.6145741878841089
0.6584723441615452
0.7023705004389815
0.746268656716418
0.7901668129938543
0.8340649692712906
0.8779631255487269
0.9218612818261633
0.9657594381035997
removing  2027  nodes
0.0
0.05197505197505198
0.10395010395010396
0.15592515592515593
0.2079002079002079
0.2598752598752599
0.31185031185031187
0.36382536382536385
0.4158004158004158
0.4677754677754678
0.5197505197505198
0.5717255717255717
0.6237006237006237
0.6756756756756757
0.7276507276507277
0.7796257796257796
0.8316008316008316
0.8835758835758836
0.9355509355509356
0.9875259875259875
removing  132  nodes


In [5]:
to_remove = []
for hyph in exp_clean.hyphaes:
    hyph.update_ts()
    if len(hyph.ts)==0:
        to_remove.append(hyph)
for hyph in to_remove:
    exp_clean.hyphaes.remove(hyph)

In [6]:
len(exp_clean.hyphaes)

3809

In [7]:
get_mother(exp_clean.hyphaes)

0.0
0.13126804935678657
0.26253609871357314
0.3938041480703597
0.5250721974271463
0.6563402467839328
0.7876082961407194
0.9188763454975059


In [8]:
def save_hyphaes(exp,path = 'Data/'):
    column_names_hyphaes = ['end','root', 'ts','mother']
    column_names_growth_info = ['hyphae','t','tp1','nodes_in_hyphae', 'segment_of_growth_t_tp1','node_list_t_tp1']
    hyphaes = pd.DataFrame(columns=column_names_hyphaes)
    growth_info = pd.DataFrame(columns = column_names_growth_info)
    for hyph in exp.hyphaes:
        new_line_hyphae=pd.DataFrame({'end' : [hyph.end.label],'root' :[hyph.root.label], 'ts': [hyph.ts],'mother' : [-1 if len(hyph.mother)==0 else hyph.mother[0].end.label]}) #index 0 for 
        #mothers need to be modified to resolve multi mother issue
        hyphaes=hyphaes.append(new_line_hyphae,ignore_index=True)
        for index in range(len(hyph.ts[:-1])):
            t = hyph.ts[index]
            tp1 = hyph.ts[index+1]
            pixels,nodes = get_pixel_growth_and_new_children(hyph,t,tp1)
            new_line_growth=pd.DataFrame({'hyphae' : [hyph.end],'t' : [t],'tp1' : [tp1],'nodes_in_hyphae' :[hyph.get_nodes_within(t)], 'segment_of_growth_t_tp1' : [pixels],'node_list_t_tp1':[nodes]})
            growth_info.append(new_line_growth,ignore_index = True)
    path='Data/'
    hyphaes.to_csv(path + f'hyphaes_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.csv')
    growth_info.to_csv(path + f'growth_info_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.csv')
    sio.savemat(path+f'hyphaes_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.mat', {name: col.values for name, col in hyphaes.items()})
    sio.savemat(path+f'growth_info_{exp.plate}_{exp.dates[0]}_{exp.dates[-1]}.mat', {name: col.values for name, col in growth_info.items()})
    return(hyphaes,growth_info)

In [ ]:
hyphs,gr_inf = save_hyphaes(exp_clean)


In [ ]:
exp_clean.save()

In [ ]:
exp_clean.pickle_save()

In [ ]:
exp_clean = pickle.load( open( f'Data/'+f"experiment_{plate}.pick", "rb" ) )

In [13]:
hyphae_with_degree4 = {}
for hyph in exp_clean.hyphaes:
    t0 = hyph.ts[-1]
    nodes,edges = hyph.get_nodes_within(t0)
    hyphae_with_degree4[hyph]=[]
    for node in nodes:
        if exp_clean.get_node(node).degree(t0)>=4:
            hyphae_with_degree4[hyph].append(node)

In [25]:
def resolve_ambiguity_two_ends(hyphaes,bottom_threshold=0.98):
    root_hyph={}
    hyphae_two_ends=[hyph for hyph in hyphaes if hyph.root.degree(hyph.ts[0])==1]
    print(len(hyphae_two_ends))
    to_remove = []
    x_boundaries = hyphaes[0].experiment.boundaries_x
    y_boundaries = hyphaes[0].experiment.boundaries_y
    for hyph in hyphae_two_ends:
        t0 = hyph.ts[0]
        if not hyph.root.pos(t0)[0]>=bottom_threshold*x_boundaries[1]:
            nodes,edges  = hyph.get_nodes_within(t0)
            mini = np.inf
            found = False
            for i,edge in enumerate(edges):
                if edge.end.degree(t0) >=4:
                    if edge.end.degree(t0)>=5:
                        print(hyph,edge.end, hyph.ts)
                    else:
                        next_edge = edges[i+1]
                        angle = np.cos((edge.orientation_end(t0,50)-next_edge.orientation_begin(t0,50))/360*2*np.pi)
                        if angle<mini:
                            found= True
                            maxi=angle
                            root_candidate = edge.end
            if found:
                root_hyph[hyph] = root_candidate
    ends = {hyph.end : hyph for hyph in hyphaes}
    for hyph in root_hyph.keys():
        if hyph.root in ends:
            ends[hyph.root].root = root_hyph[hyph]
        hyph.root = root_hyph[hyph]
    for hyph in hyphaes[0].experiment.hyphaes:
        hyph.update_ts()
    return(root_hyph)

In [23]:
for hyph in exp_clean.hyphaes:
    hyph.update_ts()

In [26]:
solve_two_ends = resolve_ambiguity_two_ends(exp_clean.hyphaes)

78
(Node(3679), Node(2354)) 2348 [4]
(Node(5931), Node(4056)) 5930 [12]
(Node(8897), Node(6619)) 8378 [21, 22]


In [15]:
hyphaes_interest = [hyph for hyph in hyphae_with_degree4.keys() if len(hyphae_with_degree4[hyph])>=1]

In [200]:
hyphae_two_ends=[hyph for hyph in exp_clean.hyphaes if hyph.root.degree(hyph.ts[0])==1]

In [83]:
weird = [hypha for hypha in hyphae_two_ends if hypha not in hyphaes_interest]

In [213]:
hyph = choice(hyphae_two_ends)
plt.close('all')

In [214]:
hyph.ts,hyph

([18, 19, 20, 21], Hyphae(7821,7832))

In [215]:
exp_clean.plot([18,20,21],[[hyph.root.label,hyph.end.label]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [68]:
hyph.end.show_source_image(15)

OSError: The file //sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE/Plate13_20200627/InitialParameters.mat does not exist.